# Pre/Post Sentences
* Still using ELMO as embedding layer
* Include pre/post sentences of the target sentences
* Perhaps the model can learn the sequential relationship between them
* The code will be pretty similar to the **ELMO.ipynb**
* Since during the time I wrote this notebook, there's only ELMO pretrained model implemented in *TF1.x*, so we return to using TensorFlow 1 

In [0]:
%tensorflow_version 1.x  # use TF1 here
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.engine import Layer
from keras.optimizers import Adam
import numpy as np

print(tf.__version__) # confirm version

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x  # use TF1 here`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.0


Using TensorFlow backend.


In [0]:
# Initialize session
sess = tf.Session()
K.set_session(sess)

## Load the data
* No stemming
* Here upload another file that only contain the label (just for upload simplicity)
* No need to pad each sentence to same length 
* No need to Tokenize

**ELMO Module will do those tasks for us !!!!!**

In [0]:
# Define some parameters
BATCH_SIZE = 64
EMBED_SIZE = 1024
MAX_LENGTH = 32

In [0]:
DATA = pd.read_csv('train_prepost.csv')
LABEL = pd.read_csv('train_label.csv')

In [0]:
#split to train and val
from sklearn.model_selection import train_test_split

DATA_train, DATA_val, LABEL_train, LABEL_val = train_test_split(DATA,LABEL, test_size=0.25, random_state=0)
print('DATA_train.shape: ', DATA_train.shape)
print('DATA_val.shape: ', DATA_val.shape)
print('LABEL_train.shape: ', LABEL_train.shape)
print('LABEL_val.shape: ', LABEL_val.shape)

DATA_train.shape:  (35150, 8)
DATA_val.shape:  (11717, 8)
LABEL_train.shape:  (35150, 6)
LABEL_val.shape:  (11717, 6)


In [0]:
Sen_train = DATA_train.loc[:,'Sentences']
# Sen_train = [' '.join(t.split()[0:MAX_LENGTH]) for t in Sen_train]  # restrict the maximum length of the sentences (if you want, it might accelerate the training/testing time)
Sen_train = np.array(Sen_train, dtype=object)[:, np.newaxis]
PreSen_train = DATA_train.loc[:,'PRE_Sen']
PreSen_train = np.array(PreSen_train, dtype=object)[:, np.newaxis]
PostSen_train = DATA_train.loc[:,'POST_Sen']
PostSen_train = np.array(PostSen_train, dtype=object)[:, np.newaxis]

#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# PrePreSen_train = DATA_train.loc[:,'PREPRE_Sen']
# PrePreSen_train = np.array(PrePreSen_train, dtype=object)[:, np.newaxis]
# PostPostSen_train = DATA_train.loc[:,'POSTPOST_Sen']
# PostPostSen_train = np.array(PostPostSen_train, dtype=object)[:, np.newaxis]
#===============================================================================

POSITION_train = DATA_train.loc[:,'POSITION':'TOTAL_LEN'].to_numpy()
LABEL_train = LABEL_train.to_numpy()


Sen_val = DATA_val.loc[:,'Sentences']
# Sen_val = [' '.join(t.split()[0:MAX_LENGTH]) for t in Sen_val]  # restrict the maximum length of the sentences (if you need)
Sen_val = np.array(Sen_val, dtype=object)[:, np.newaxis]
PreSen_val = DATA_val.loc[:,'PRE_Sen']
PreSen_val = np.array(PreSen_val, dtype=object)[:, np.newaxis]
PostSen_val = DATA_val.loc[:,'POST_Sen']
PostSen_val = np.array(PostSen_val, dtype=object)[:, np.newaxis]

#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# PrePreSen_val = DATA_val.loc[:,'PREPRE_Sen']
# PrePreSen_val = np.array(PrePreSen_val, dtype=object)[:, np.newaxis]
# PostPostSen_val = DATA_val.loc[:,'POSTPOST_Sen']
# PostPostSen_val = np.array(PostPostSen_val, dtype=object)[:, np.newaxis]
#===============================================================================

POSITION_val = DATA_val.loc[:,'POSITION':'TOTAL_LEN'].to_numpy()
LABEL_val = LABEL_val.to_numpy()

print(Sen_train.shape, Sen_val.shape)
print(PreSen_train.shape, PreSen_val.shape)
print(PostSen_train.shape, PostSen_val.shape)
print(POSITION_train.shape, POSITION_val.shape)
print(LABEL_train.shape, LABEL_val.shape)
#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# print(PrePreSen_train.shape, PrePreSen_val.shape)
# print(PostPostSen_train.shape, PostPostSen_val.shape)
#===============================================================================

(35150, 1) (11717, 1)
(35150, 1) (11717, 1)
(35150, 1) (11717, 1)
(35150, 2) (11717, 2)
(35150, 6) (11717, 6)


## Define the model
Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)
* Three ELMO, for Target Sentences, Pre Sentences and Post Sentences
* Maybe change to only one ELMO?

In [0]:
K.clear_session()

In [0]:
# Function to calculate F1_score
def F1_score(y_true, y_pred):
  DTYPE = tf.float32
  THRESHOLD = 0.5

  y_pred = tf.cast(y_pred > THRESHOLD, DTYPE) 

  true_positives = tf.math.count_nonzero(tf.math.logical_and(tf.math.equal(y_pred,1.0), tf.math.equal(y_true,1.0)), axis=0)
  false_positives = tf.math.count_nonzero(tf.math.logical_and(tf.math.equal(y_pred,1.0), tf.math.equal(y_true,0.0)), axis=0)
  false_negatives = tf.math.count_nonzero(tf.math.logical_and(tf.math.equal(y_pred,0.0), tf.math.equal(y_true,1.0)), axis=0)

  TP = tf.math.reduce_sum(tf.cast(true_positives, DTYPE), axis=0)
  FP = tf.math.reduce_sum(tf.cast(false_positives, DTYPE), axis=0)
  FN = tf.math.reduce_sum(tf.cast(false_negatives, DTYPE), axis=0)

  precision = tf.math.divide_no_nan(TP, TP+FP)
  recall = tf.math.divide_no_nan(TP, TP+FN)

  F1 = tf.math.divide_no_nan(2 * (precision * recall) , (precision + recall))
  return F1

### ELMO as word embedding
* Using pretrained ELMO as word embedding layer
* The weight for combining the output of each layer of the ELMO is trainable(Only 4 variables) [REF](https://tfhub.dev/google/elmo/3)
* [Source](https://github.com/strongio/keras-elmo/blob/master/Elmo%20Keras.ipynb)
* Return size =  ``` [Batch_size(None), max_length(None), 1024] ```

(The ELMO module will automatcally detect the max_length of each input batch)

In [0]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
      self.dimensions = 1024
      self.trainable=True
      super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
      self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                              name="{}_module".format(self.name))

      self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
      super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
      result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                    as_dict=True,
                    signature='default',
                    )['elmo']
      return result
      
    def compute_output_shape(self, input_shape):
      return (input_shape[0], None, self.dimensions)

### ELMO as sentences embedding
* Same as above, but the return is **default**. The model will return the mean-pooling of all words embedding result
* Return size =  ``` [Batch_size(None), 1024] ```



In [0]:
class ElmoSentenceEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
      self.dimensions = 1024
      self.trainable=True
      super(ElmoSentenceEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
      self.elmo = hub.Module('https://tfhub.dev/google/elmo/3', trainable=self.trainable,
                              name="{}_module".format(self.name))

      self.trainable_weights += tf.trainable_variables(scope="^{}_module/.*".format(self.name))
      super(ElmoSentenceEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
      result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                    as_dict=True,
                    signature='default',
                    )['default']
      return result
      
    def compute_output_shape(self, input_shape):
      return (input_shape[0], self.dimensions)

In [0]:
def concatresult(inputs):
  inputs_1_expand = K.expand_dims(inputs[0], 1)
  inputs_2_expand = K.expand_dims(inputs[1], 1)
  inputs_3_expand = K.expand_dims(inputs[2], 1)
  result = K.concatenate([inputs_1_expand,inputs_2_expand,inputs_3_expand], axis=1)
  
  #%% see blocks below - 2 Pre/Post Sentences
  #=============================================================================
  # inputs_4_expand = K.expand_dims(inputs[3], 1)
  # inputs_5_expand = K.expand_dims(inputs[4], 1)
  # result = K.concatenate([inputs_1_expand,inputs_2_expand,inputs_3_expand,inputs_4_expand,inputs_5_expand], axis=1)
  #=============================================================================
  return result

In [0]:
# use to concat the position in the input of the word vector, do not use in the final version of the modle
def repeat_and_concatenate(inputs):
    embed, position = inputs
    # Repeat 2D vectors
    position_repeat = K.tile(K.expand_dims(position, 1), [1, K.shape(embed)[1], 1])
    # Concatenate feature-wise
    return K.concatenate([embed, position_repeat], axis=-1)

### Model_1
* Using three ELMO as word embedding, then input to Bi_GRU as sentences embedding
* Later, using a Dense layer to reduce the length of **Pre_Sentence** and **Post_Sentence**
* Concat all vectors (include some features other than text input) as feature vector of the target sentence, then using a Dense layer as classifier

In [0]:
# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), name='Target_sentences', dtype=tf.string)
  input_pre = layers.Input(shape=(1,), name='Pre_sentences', dtype=tf.string)
  input_post = layers.Input(shape=(1,), name='Post_sentences', dtype=tf.string)
  input_position = layers.Input(shape=(2,), name='Sentence_position', dtype=tf.float32)

  embedding_target = ElmoEmbeddingLayer(name="ElmoEmbed_target")(input_text)
  # test_position = layers.Lambda(repeat_and_concatenate, name='Merge_position')([embedding_target, input_position])  #version2: add position in the embedding of the words
  GRU_target = layers.Bidirectional(layers.GRU(256, dropout=0.5, return_sequences=False), name='BiGRU_target')(embedding_target)

  embedding_pre = ElmoEmbeddingLayer(name="ElmoEmbed_pre")(input_pre)
  GRU_pre = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_pre')(embedding_pre)
  embedding_post = ElmoEmbeddingLayer(name="ElmoEmbed_post")(input_post)
  GRU_post = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_post')(embedding_post)
  
  pre = layers.Dense(20, activation='sigmoid')(GRU_pre)
  post = layers.Dense(20, activation='sigmoid')(GRU_post)

  concat = layers.concatenate([pre, GRU_target, post, input_position], name='Merge')

  pred = layers.Dense(6, activation='sigmoid')(concat)

  model = Model(inputs=[input_text,input_pre,input_post,input_position], outputs=pred)
  adam = Adam(lr=0.001)

  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[F1_score])
  model.summary()
  
  return model

model = build_model()
# save the weight for later retraining the whole module
weights = model.get_weights()

### Model_2
* Using Dense layer as kind of summary of the output of ELMOSentenceEmbedding
* Later, using another Bi_GRU as an encoder to encode the \[Pre_Sentence, Target_Sentence, Post_Sentence\]
* The output of the encoder is the summary of these three sentences which bias to the target sentence
* Concatenate with some non-text input to form the final feature vector of the target sentence, then using a Dense layer as classifier

In [0]:
# Function to build model
def build_model(): 
  input_text = layers.Input(shape=(1,), name='Target_sentences', dtype=tf.string)
  input_pre = layers.Input(shape=(1,), name='Pre_sentences', dtype=tf.string)
  input_post = layers.Input(shape=(1,), name='Post_sentences', dtype=tf.string)
  input_position = layers.Input(shape=(2,), name='Sentence_position', dtype=tf.float32)

  embedding_target = ElmoEmbeddingLayer(name="ElmoEmbed_target")(input_text)
  # target_withposition = layers.Lambda(repeat_and_concatenate, name='Merge_position')([embedding_target, input_position]) #version2: add position in the embedding of the words
  GRU_target = layers.Bidirectional(layers.GRU(128, dropout=0.5, return_sequences=False), name='BiGRU_target')(embedding_target)

  embedding_pre = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_pre")(input_pre)
  GRU_pre = layers.Dense(256, activation='sigmoid')(embedding_pre)
  embedding_post = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_post")(input_post)
  GRU_post = layers.Dense(256, activation='sigmoid')(embedding_post)
  
  #%% we can also encode the sentences same with Model_1, but since this will result in lots of layers of Bi_GRU which will cost too much time to train
  # embedding_pre = ElmoEmbeddingLayer(name="ElmoEmbed_pre")(input_pre)
  # GRU_pre = layers.Bidirectional(layers.GRU(256, dropout=0.5, return_sequences=False), name='BiGRU_pre')(embedding_pre)
  # embedding_post = ElmoEmbeddingLayer(name="ElmoEmbed_post")(input_post)
  # GRU_post = layers.Bidirectional(layers.GRU(256, dropout=0.5, return_sequences=False), name='BiGRU_post')(embedding_post)

  concat =  layers.Lambda(concatresult, name='Merge_Sentences')([GRU_pre, GRU_target, GRU_post])
  Summary = layers.Bidirectional(layers.GRU(128, dropout=0.5, return_sequences=False), name='BiGRU_Summary')(concat)

  addposition = layers.concatenate([Summary, input_position], name='Merge_position')
  pred = layers.Dense(6, activation='sigmoid')(addposition)

  model = Model(inputs=[input_text,input_pre,input_post,input_position], outputs=pred)
  adam = Adam(lr=0.001)

  model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[F1_score])
  model.summary()
  
  return model

model = build_model()
# save the weight for later retraining the whole module
weights = model.get_weights()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Pre_sentences (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
Target_sentences (InputLayer)   (None, 1)            0                                            
__________________________________________________________________________________________________
Post_sentences (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
ElmoEmbed_pre (ElmoSentenceEmbe (None, 1024)         4           Pre_sentences[0][0]              
____________________________________________________________________________________________

## Training

In [0]:
# Monitor the performance
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_F1_score', patience=3, mode='max', restore_best_weights=True)

history = model.fit([Sen_train,PreSen_train,PostSen_train,POSITION_train], LABEL_train,
          validation_data=([Sen_val,PreSen_val,PostSen_val,POSITION_val], LABEL_val),
          epochs=20, batch_size=BATCH_SIZE,
          callbacks=[early_stopping])

## Evaluate

In [0]:
result = model.predict([Sen_val,PreSen_val,PostSen_val,POSITION_val,PrePreSen_val,PostPostSen_val])

print(result.shape)
print(result[-5:-1])

(11717, 6)
[[0.00757217 0.06803593 0.9603202  0.03049085 0.00372973 0.00290501]
 [0.00969911 0.5053471  0.1252133  0.18213439 0.39775497 0.04539654]
 [0.9133489  0.10974118 0.01804778 0.0061903  0.00242183 0.01721632]
 [0.00727654 0.57898104 0.471901   0.07808304 0.01865387 0.00378391]]


In [0]:
from sklearn.metrics import f1_score

greater = (result>=0.5).astype(int)
print(LABEL_val.shape)
print(greater.shape)
print(f1_score(LABEL_val, greater, average='micro'))

(11717, 6)
(11717, 6)
0.6935446282917734


Examine the validation result
* Also store the Sentence so that we can examine under which condition of the sentences our prediction will go wrong

In [0]:
df1 = pd.DataFrame(result)
df2 = pd.DataFrame(LABEL_val)
df3 = pd.DataFrame(Sen_val) # store the sentences too

In [0]:
val_result = pd.concat([df1,df2, df3], axis=1)
val_result.head()

,0,1,2,3,4,5,0,1,2,3,4,5,0
0,0.009559,0.146008,0.229192,0.745078,0.305471,0.012025,0,1,0,1,0,0,We demonstrate some of the useful properties a...
1,0.866730,0.101646,0.013343,0.003884,0.006394,0.010817,1,0,0,0,0,0,As the technologies used within NAME advance i...
2,0.647780,0.408719,0.027728,0.006821,0.007124,0.013341,1,0,0,0,0,0,Yet these methods often fail to capture the mo...
3,0.724096,0.466831,0.005982,0.004289,0.005403,0.010023,1,0,0,0,0,0,Generating novel yet realistic images of perso...
4,0.011359,0.023873,0.229095,0.829455,0.050550,0.001132,0,0,1,0,0,0,Evaluations using separate genre classifiers s...


In [0]:
val_result.to_csv("val_result.csv", index=False)

### Refit on the whole data with around 5 epochs

In [0]:
Sen = DATA.loc[:,'Sentences']
# Sen = [' '.join(t.split()[0:MAX_LENGTH]) for t in Sen]  # restrict the maximum length of the sentences (if you need)
Sen = np.array(Sen, dtype=object)[:, np.newaxis]
PreSen = DATA.loc[:,'PRE_Sen']
PreSen = np.array(PreSen, dtype=object)[:, np.newaxis]
PostSen = DATA.loc[:,'POST_Sen']
PostSen = np.array(PostSen, dtype=object)[:, np.newaxis]

#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# PrePreSen = DATA.loc[:,'PREPRE_Sen']
# PrePreSen = np.array(PrePreSen, dtype=object)[:, np.newaxis]
# PostPostSen = DATA.loc[:,'POSTPOST_Sen']
# PostPostSen = np.array(PostPostSen, dtype=object)[:, np.newaxis]
#===============================================================================

POSITION = DATA.loc[:,'POSITION':'TOTAL_LEN'].to_numpy()
LABEL = LABEL.to_numpy()

In [0]:
print(Sen.shape)
print(PreSen.shape)
print(PostSen.shape)
#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# print(PrePreSen.shape)
# print(PostPostSen.shape)
#===============================================================================
print(POSITION.shape)
print(LABEL.shape)

(46867, 1)
(46867, 1)
(46867, 1)
(46867, 2)
(46867, 6)


In [0]:
# reload the model weight
model.set_weights(weights)
# train with best epoch = 5
history = model.fit([Sen,PreSen,PostSen,POSITION], LABEL, epochs=5, batch_size=BATCH_SIZE)

Epoch 1/5
46867/46867 [==============================] - 793s 17ms/step - loss: 0.3094 - F1_score: 0.5993
Epoch 2/5
46867/46867 [==============================] - 796s 17ms/step - loss: 0.2764 - F1_score: 0.6675
Epoch 3/5
46867/46867 [==============================] - 803s 17ms/step - loss: 0.2660 - F1_score: 0.6833
Epoch 4/5
46867/46867 [==============================] - 771s 16ms/step - loss: 0.2574 - F1_score: 0.6962
Epoch 5/5
46867/46867 [==============================] - 766s 16ms/step - loss: 0.2502 - F1_score: 0.7057


## Predict

In [0]:
TEST_DATA = pd.read_csv('test_prepost.csv')

In [0]:
Test_Sen = TEST_DATA.loc[:,'Sentences']
# Test_Sen = [' '.join(t.split()[0:MAX_LENGTH]) for t in Test_Sen]  # restrict the maximum length of the sentences (if you need)
Test_Sen = np.array(Test_Sen, dtype=object)[:, np.newaxis]
Test_PreSen = TEST_DATA.loc[:,'PRE_Sen']
Test_PreSen = np.array(Test_PreSen, dtype=object)[:, np.newaxis]
Test_PostSen = TEST_DATA.loc[:,'POST_Sen']
Test_PostSen = np.array(Test_PostSen, dtype=object)[:, np.newaxis]

#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# Test_PrePreSen = TEST_DATA.loc[:,'PREPRE_Sen']
# Test_PrePreSen = np.array(Test_PrePreSen, dtype=object)[:, np.newaxis]
# Test_PostPostSen = TEST_DATA.loc[:,'POSTPOST_Sen']
# Test_PostPostSen = np.array(Test_PostPostSen, dtype=object)[:, np.newaxis]
#===============================================================================

Test_POSITION = TEST_DATA.loc[:,'POSITION':'TOTAL_LEN'].to_numpy()

In [0]:
print(Test_Sen.shape)
print(Test_PreSen.shape)
print(Test_PostSen.shape)
#%% see blocks below - 2 Pre/Post Sentences
#===============================================================================
# print(Test_PrePreSen.shape)
# print(Test_PostPostSen.shape)
#===============================================================================
print(Test_POSITION.shape)

(131782, 1)
(131782, 1)
(131782, 1)
(131782, 2)


In [0]:
result = model.predict([Test_Sen,Test_PreSen,Test_PostSen,Test_POSITION])

print(result.shape)
print(result[-5:-1])

(131782, 6)
[[0.859565   0.05170634 0.00781983 0.00469366 0.00302225 0.00627455]
 [0.92724466 0.01253507 0.00733677 0.01177165 0.00392583 0.00221708]
 [0.5364166  0.11997902 0.00786909 0.00517094 0.00275862 0.00648162]
 [0.00192121 0.8123803  0.408198   0.07168016 0.04696891 0.00132227]]


In [0]:
RESULT = pd.DataFrame(result, columns=['BACKGROUND','OBJECTIVES','METHODS','RESULTS','CONCLUSIONS','OTHERS'])
RESULT.head()

,BACKGROUND,OBJECTIVES,METHODS,RESULTS,CONCLUSIONS,OTHERS
0,0.993274,0.014937,0.002228,0.001413,0.000702,0.003719
1,0.870303,0.256270,0.020141,0.005964,0.002896,0.010243
2,0.948224,0.120969,0.030201,0.003545,0.002351,0.005209
3,0.007879,0.446890,0.832253,0.029567,0.014735,0.002820
4,0.005733,0.051138,0.955495,0.056679,0.009260,0.003050


In [0]:
RESULT.to_csv("test_result.csv", index=False)

In [0]:
# save the weight
# model.save_weights('ELMO_withprepostsentences.h5')

## Load weight
If we got new input testing data, we can initialize the model first then load the weight we have saved to do the prediction
* need to initialize the model beforehand since we only save the weight but not the model structure

In [0]:
# model = build_model()
# model.load_weights('ELMO_withprepostsentences.h5')

# 2 Pre/Post Sentences
* Same as above, but add more pre/post sentences of the target sentences
* Perhaps the model can learn more from more context of the target sentences
* you can find some commented line in the above with the 
  - prepre
  - postpost 

  that is the data of second pre/post sentences

## Adjust the model
### Model_1


In [0]:
input_text = layers.Input(shape=(1,), name='Target_sentences', dtype=tf.string)
input_pre = layers.Input(shape=(1,), name='Pre_sentences', dtype=tf.string)
input_post = layers.Input(shape=(1,), name='Post_sentences', dtype=tf.string)
#%% added
#===============================================================================
input_prepre = layers.Input(shape=(1,), name='PrePre_sentences', dtype=tf.string)
input_postpost = layers.Input(shape=(1,), name='PostPost_sentences', dtype=tf.string)
#===============================================================================
input_position = layers.Input(shape=(2,), name='Sentence_position', dtype=tf.float32)

embedding_target = ElmoEmbeddingLayer(name="ElmoEmbed_target")(input_text)
GRU_target = layers.Bidirectional(layers.GRU(256, dropout=0.5, return_sequences=False), name='BiGRU_target')(embedding_target)

embedding_pre = ElmoEmbeddingLayer(name="ElmoEmbed_pre")(input_pre)
GRU_pre = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_pre')(embedding_pre)
embedding_post = ElmoEmbeddingLayer(name="ElmoEmbed_post")(input_post)
GRU_post = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_post')(embedding_post)
#%% added
#===============================================================================
embedding_prepre = ElmoEmbeddingLayer(name="ElmoEmbed_prepre")(input_prepre)
GRU_prepre = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_prepre')(embedding_prepre)
embedding_postpost = ElmoEmbeddingLayer(name="ElmoEmbed_postpost")(input_postpost)
GRU_postpost = layers.Bidirectional(layers.GRU(64, dropout=0.5, return_sequences=False), name='BiGRU_postpost')(embedding_postpost)
#===============================================================================

pre = layers.Dense(20, activation='sigmoid')(GRU_pre)
post = layers.Dense(20, activation='sigmoid')(GRU_post)
#%% added
#===============================================================================
prepre = layers.Dense(20, activation='sigmoid')(GRU_prepre)
postpost = layers.Dense(20, activation='sigmoid')(GRU_postpost)
#===============================================================================

#%% add prepre postpost
concat = layers.concatenate([prepre, pre, GRU_target, post, postpost, input_position], name='Merge')

pred = layers.Dense(6, activation='sigmoid')(concat)
#%% add prepre postpost input
model = Model(inputs=[input_text,input_pre,input_post,input_position,input_prepre,input_postpost], outputs=pred)

### Model_2

In [0]:
input_text = layers.Input(shape=(1,), name='Target_sentences', dtype=tf.string)
input_pre = layers.Input(shape=(1,), name='Pre_sentences', dtype=tf.string)
input_post = layers.Input(shape=(1,), name='Post_sentences', dtype=tf.string)
#%% added
#===============================================================================
input_prepre = layers.Input(shape=(1,), name='PrePre_sentences', dtype=tf.string)
input_postpost = layers.Input(shape=(1,), name='PostPost_sentences', dtype=tf.string)
#===============================================================================
input_position = layers.Input(shape=(2,), name='Sentence_position', dtype=tf.float32)

embedding_target = ElmoEmbeddingLayer(name="ElmoEmbed_target")(input_text)
GRU_target = layers.Bidirectional(layers.GRU(128, dropout=0.5, return_sequences=False), name='BiGRU_target')(embedding_target)
embedding_pre = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_pre")(input_pre)
GRU_pre = layers.Dense(256, activation='sigmoid')(embedding_pre)
embedding_post = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_post")(input_post)
GRU_post = layers.Dense(256, activation='sigmoid')(embedding_post)

#%% added
#===============================================================================
embedding_prepre = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_prepre")(input_prepre)
GRU_prepre = layers.Dense(256, activation='sigmoid')(embedding_prepre)
embedding_postpost = ElmoSentenceEmbeddingLayer(name="ElmoEmbed_postpost")(input_postpost)
GRU_postpost = layers.Dense(256, activation='sigmoid')(embedding_postpost)
#===============================================================================

#%% add prepre postpost GRU result
concat =  layers.Lambda(concatresult, name='Merge_Sentences')([GRU_prepre, GRU_pre, GRU_target, GRU_post, GRU_postpost])
Summary = layers.Bidirectional(layers.GRU(128, dropout=0.5, return_sequences=False), name='BiGRU_Summary')(concat)

addposition = layers.concatenate([Summary, input_position], name='Merge_position')
pred = layers.Dense(6, activation='sigmoid')(addposition)

#%% add prepre postpost input
model = Model(inputs=[input_text,input_pre,input_post,input_position,input_prepre,input_postpost], outputs=pred)

## Input becomes
```
[Sen,PreSen,PostSen,POSITION,PrePreSen,PostPostSen]
```

**Remember to include the PrePre/PostPost Input !!!!!!!!!**
